In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

In [53]:
df_train_stops = pd.read_csv('http://web.mta.info/developers/data/nyct/subway/Stations.csv')

In [23]:
len(df_train_stops['Stop Name'].unique())

376

### Making a df from turnstile data

In [24]:
dates = ['170624', '170617', '170610', '170603', '170527', '170520', '170513', '170506', '160625', '160618', '160611', '160604', '160528', '160521', '160514', '160507']

chronological_dates = list(reversed(dates))

In [25]:
turnstile_df = pd.DataFrame()

for date in chronological_dates:
    base = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_'
    url = base + date + '.txt'
    print(url)
    df = pd.read_csv(url)
    turnstile_df = pd.concat([turnstile_df, df], ignore_index=True)

http://web.mta.info/developers/data/nyct/turnstile/turnstile_160507.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160514.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160521.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160528.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160604.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160611.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160618.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160625.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_170506.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_170513.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_170520.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_170527.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_170603.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_170

In [ ]:
for item in turnstile_df['STATION'].str.lower().unique():
    if item not in df_train_stops['Stop Name'].str.lower().unique():
       # print(item)

In [167]:
df_SS = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_170916.txt')
                 #http://web.mta.info/developers/data/nyct/subway/StationEntrances.csv')


In [168]:
turnstile_df.rename(columns = {turnstile_df.columns[-1]:'EXITS'}, inplace=True)
df_SS.rename(columns = {df_SS.columns[-1]:'EXITS'}, inplace=True) 

### Get and Compute the total number of people in and out of each station, each day.

In [333]:
def ppl_per_int_turnst(datafrm, dt_start='06/09/2016', dt_end = '06/11/2016'): 

    """function to subtract the entries and exits collected at the starting hour for the day (i.e., 00:00:00 hrs) from the entries 
    and exits collected during the next collection time (i.e. 04:00:00 hrs) for each turnstile in MTA.
    ----------
    dt_start = list start day as string in format mm/dd/yyyy  
    dt_end = end day as string in format mm/dd/yyyy
    datafrm= dataframe from MTA turnstile data"""
    #dt = dt_start # needs to loop through a range of dates
    #if # find the hours for each row (TIME) and put into a list, then get max and min  & (datafrm.TIME == '00:00:00'), :])
    from datetime import datetime
    from datetime import timedelta
    
    def get_dates(start,stop):
        """ make a list of dates in format mm/dd/yyyy
        ------------------
        start = list start day as string in format mm/dd/yyyy
        end day as string in format mm/dd/yyyy"""
        days = []
        weekdays = []
        date_format = '%m/%d/%Y'
        date_ob_st = datetime.strptime(start,date_format)
        date_ob_end = datetime.strptime(stop,date_format)
        num_of_days_ob = str(date_ob_end - date_ob_st) 
        num_of_days = int(num_of_days_ob[0]) #total number of days to include
        for day in  range(1,num_of_days+1):
            one_day = date_ob_st + timedelta (days=day) 
            days.append(datetime.strftime(one_day,date_format))
            weekdays.append(one_day.weekday())
        return days
    
    dates = get_dates(dt_start,dt_end)
    
    for day in dates: 
        day1 = datafrm.loc[(datafrm.DATE == day ),['DATE','STATION','LINENAME','SCP','TIME','ENTRIES','EXITS']]
        lins = (day1['LINENAME'].unique())
        stns = (day1['STATION'].unique())
        turnstile_int = pd.DataFrame(index=range(len(datafrm)),columns=['DATE','STATION','LINENAME','ID','HOUR','ENTRIES','EXITS','WEEKDAY'])
        rowDF = 0 
        for ln in lins :
            lines = pd.DataFrame(columns=day1.columns)
            lines = day1.loc[(day1.LINENAME==ln)]
            for stn in stns:
                st = pd.DataFrame(columns=lines.columns)
                st = lines.loc[(day1.STATION==stn)]
                turnstiles = st['SCP'].unique()
                for turnstile in turnstiles:
                    turnstile_data = pd.DataFrame(columns=st.columns)
                    turnstile_data = st.loc[(st.SCP==turnstile)] 
                    for row in range(len(turnstile_data)-1) : # subtract counts from the previous count (Entries/Exits) and put into dataframe called turnstile_int
                        # if datetime.strptime(turnstile_data.TIME.iloc[row], '%H:%M:%S') < datetime.strptime(turnstile_data.TIME.iloc[row+1], '%H:%M:%S'):               
                        turnstile_int.DATE.iloc[rowDF] = turnstile_data.DATE.iloc[row]
                        turnstile_int.STATION.iloc[rowDF] = turnstile_data.STATION.iloc[row]
                        turnstile_int.ID.iloc[rowDF] = turnstile_data.SCP.iloc[row]
                        turnstile_int.LINENAME.iloc[rowDF] = turnstile_data.LINENAME.iloc[row]
                        turnstile_int.HOUR.iloc[rowDF] = turnstile_data.TIME.iloc[row+1]
                        wd = datetime.strptime(day,"%m/%d/%Y")
                        turnstile_int.WEEKDAY.iloc[rowDF] = wd.weekday()

                        if int(turnstile_data.ENTRIES.iloc[row+1]) == 0 : # if turnstyle resets, don't include a number for that interval
                            turnstile_int.ENTRIES.iloc[rowDF] = np.nan
                        else: 
                            turnstile_int.ENTRIES.iloc[rowDF] = abs(int(turnstile_data.ENTRIES.iloc[row+1]) - int(turnstile_data.ENTRIES.iloc[row]))  #subtract numb from start of day from end of day for each turnstyle

                        if int(turnstile_data.EXITS.iloc[row+1]) == 0 :# if turnstyle resets, don't include a number for that interval
                            turnstile_int.EXITS.iloc[rowDF] = np.nan      
                        else:                   
                            turnstile_int.EXITS.iloc[rowDF] = abs(int(turnstile_data.EXITS.iloc[row+1]) - int(turnstile_data.EXITS.iloc[row]))  #subtract numb from start of day from end of day for each turnstyle

                        rowDF = rowDF + 1 
    turnstile_int.dropna(subset=['DATE'], inplace = True)                    
    return turnstile_int

In [ ]:
shinyhappypeople = ppl_per_int_turnst(df_SS, '09/14/2017','9/15/2017')
shinyhappypeople


In [280]:
import pickle
# to save a pickle file
path = '/Users/ada/Dropbox/metis/MTABenson_metis/pklfiles/'
with open(path + 'shinyhappypeople.pkl', 'wb') as picklefile:
    pickle.dump(shinyhappypeople, picklefile)

In [ ]:
# to open a pickle file
with open(path + "subset_df.pkl", 'rb') as picklefile: 
    test = pickle.load(picklefile)